# Import Libraries

In [1]:
from swarmintelligence import SlimeMouldAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/SMA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_iteration': 100, 'z': 0.3}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
866.0839233076948  |  2396.7203053596963  |  1660.4458429146562
838.921371836062  |  2367.7506045003297  |  1778.105902885669
930.1268857315627  |  2387.5192854195047  |  1771.0567294652424
809.0272821715043  |  2213.600049123271  |  1782.0047674370292
854.1378822454412  |  2326.562517235211  |  1738.0352004182032
814.6973941037899  |  2226.8534924130754  |  1781.949029717393
776.7880280878883  |  2328.817988951374  |  1701.6253343668623
908.4688899217804  |  2429.758835789239  |  1737.0428422596092
782.5709416071294  |  2344.787293602499  |  1773.6112752178747
895.9504339056008  |  2222.3841955868143  |  1774.4887062956604
913.654508688451  |  2417.8598250931527  |  1625.624981199955
900.1611072104902  |  2381.484061961298  |  1662.1508325693435
822.6724018432036  |  2409.732138754595  |  1709.1244462941104
930.506219472519  |  2406.0426808851516  |  1758.185808219978
874.4679449212138  |  2329.858752996839  |  1744.3469912779678
925.5221766938911  

734.1293125340219  |  2189.5170537402673  |  2095.3782138244837
786.1062875374504  |  2017.6845734980893  |  2038.373394062458
817.3818417535485  |  2320.7152734238516  |  2135.7868171677383
857.1780719986916  |  2344.5681435159727  |  2133.758217799786
1000.3005289810715  |  2050.887697570302  |  1993.6720655866413
888.6202642761534  |  2186.5404968030193  |  2084.6299116058544
715.4566781452313  |  1919.7671385390101  |  2013.2769364110154
993.0268744830651  |  2392.383595690778  |  2120.7262567666144
992.1642281417135  |  2218.6207377074006  |  2161.3446710416524
756.9783458237004  |  2335.8801395970368  |  2169.7011738681317
883.0867162175095  |  2253.4503151880863  |  2010.8815086644845
905.6197268865578  |  1917.6057355592698  |  1873.3609466715643
996.3774249753938  |  2327.8775631079643  |  2173.7616698829124
976.785836307234  |  2315.772981083933  |  2148.067378109457
782.1091737238471  |  2205.6024148858155  |  2174.107555390003
999.4444823555258  |  2173.0446045398685  |  21

2856.9617018171843  |  2916.9546933626234  |  2800.63435722084
2829.152426140428  |  2931.333204278032  |  2886.5571993246817
2632.8961844360274  |  2722.186623240715  |  2847.012688780011
2916.705887921627  |  2908.407758611006  |  2699.652227574696
2884.851929737311  |  2725.2850453779597  |  2917.59141883994
2613.7280442499164  |  2813.6718918712345  |  2867.398303805122
2868.1964130052165  |  2930.850888068557  |  2519.7155174390687
2784.1756518645725  |  2520.390906343663  |  2839.914482810132
2854.6534568135066  |  2541.5999899845665  |  2729.2861644933687
2713.9186666242267  |  2924.9405600105483  |  2897.510504469442
2852.366821649681  |  2802.395055819344  |  2822.031534896055
2939.0082868228474  |  2664.176120961867  |  2881.512465450845
2868.1149372407726  |  2839.03185185349  |  2859.9637509356535
2509.4579951653627  |  2928.7797223784246  |  2923.91989468479
2789.564630324698  |  2852.9447300909187  |  2806.539812460884
2814.003736034752  |  2898.7978613904115  |  2824.332

3331.5292914391953  |  1685.069353924956  |  2379.84052043785
3348.9945147490635  |  1774.0756721288826  |  2619.5686181816573
3355.9989933622755  |  1932.1475140656926  |  2283.532320492466
3241.749606430472  |  1756.2772321483444  |  2559.089108713574
3244.3575306074445  |  1882.0020479782952  |  2499.482607988063
3356.848638962839  |  1722.2501799910972  |  2251.783387694056
3264.0408884009394  |  1943.4791787500537  |  2638.820621349259
3117.8353259999885  |  1901.3397801987276  |  2176.9245403986733
3267.582577008185  |  1926.8267465301428  |  2568.5966940758744
3106.4604733592314  |  1992.0564810014203  |  2529.2832884562076
3337.9798252231744  |  1975.9113349349545  |  2399.875093801322
3105.747214018782  |  1916.4971467584971  |  2411.201601298617
3244.7947616707297  |  1692.4170825213273  |  2569.8302838335694
2941.9266296440173  |  1822.8816237898577  |  2620.792970258433
3243.6688784554494  |  1697.581187900484  |  2572.7825397198476
3042.403675112426  |  1953.6920737683317 

1394.8303544374953  |  5162.845282469258  |  1673.048286724345
1486.4505145339758  |  4994.468349101859  |  1566.5546803868588
1548.6552001984016  |  5079.00622858717  |  1607.4643717370454
1451.9588879179018  |  5167.792968744398  |  1720.104297501871
1400.0432789347556  |  5125.581772286903  |  1534.8808089107638
1518.1830773187378  |  5078.777154950989  |  1737.880279603397
1434.0208272331054  |  5152.595968731199  |  1694.4301489040436
1406.5298663722383  |  5109.238202407092  |  1566.6770340153932
1252.6108843939905  |  5154.171471253983  |  1705.26430735548
1646.0326064163087  |  5083.686949108585  |  1648.190591256696
1544.8641163925672  |  5182.822893588656  |  1682.7526694258904
1390.11931357728  |  5113.940940962937  |  1745.8361595283513
1627.3021134931473  |  5199.370217198889  |  1678.093641956743
1586.2866574410266  |  4979.961557378319  |  1688.0093891247006
1581.3730588648161  |  5150.782789720073  |  1748.5146643632118
1288.5982162553585  |  5168.457645974027  |  1645.

5579.746185692864  |  5567.452271412078  |  1557.7688110277054
5638.9278163974495  |  5234.783788113488  |  1403.288266670558
5527.017062674411  |  5627.555998115589  |  1416.0957475053501
5550.452069281602  |  5585.421052476312  |  1535.8820175425649
5582.585248162233  |  5224.097414725924  |  1549.8485662122303
5589.953604630073  |  5553.277989146029  |  1569.715727808344
5635.665640731375  |  5466.514701987815  |  1492.6779010923506
5588.125939376971  |  5233.823795375478  |  1483.5558553359494
5446.497052716423  |  5586.413087084506  |  1537.481624066534
5542.0711527184885  |  5625.766870501703  |  1541.6749346249362
5530.326770322069  |  5490.440575300861  |  1587.173288894977
5640.054531490594  |  5628.33725381047  |  1361.0825413259706
5477.5639021171555  |  5627.740083108851  |  1435.3695458600316
5638.702881770834  |  5613.633848506886  |  1546.8020675115463
5548.386592326718  |  5504.270241013619  |  1459.7798261873827
5601.328706229829  |  5481.943244038834  |  1517.44025809

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[119, 176]","[99, 168]","[127, 186]",879.105978,2377.766384,1733.958133,...,"[[127, 255, 255, 255, 255, 255, 255, 255, 255,...","[866.0839233076948, 838.921371836062, 930.1268...","[2396.7203053596963, 2367.7506045003297, 2387....","[1660.4458429146562, 1778.105902885669, 1771.0...","[2.537398338317871, 2.495837926864624, 2.35236...","[2483.558848063151, 2490.110768636068, 2347.79...","[49.83531727663777, 49.901009695556944, 48.454...","[14.18005905783174, 14.16861694439872, 14.4241...","[0.7529692499247611, 0.767250174310658, 0.7783...","[0.9331985762775257, 0.9340237704780258, 0.939..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[83, 150, 189]","[68, 138, 182]","[93, 147, 189]",932.104492,2455.862827,1809.108097,...,"[[147, 255, 255, 255, 255, 255, 255, 255, 255,...","[944.1354049049486, 899.0891885333431, 965.095...","[2435.9367463851177, 2509.791025188011, 2347.8...","[1812.921822118114, 1797.3832997773438, 1841.6...","[4.161643028259277, 3.3342862129211426, 3.7254...","[2056.5228551228843, 2214.579126993815, 2167.1...","[45.34890136621707, 47.05931498644891, 46.5531...","[14.999468204734354, 14.677891586120662, 14.77...","[0.75641447008145, 0.7797427414709297, 0.76199...","[0.9472414874614944, 0.9436719199998705, 0.946..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[70, 128, 164, 198]","[49, 104, 153, 197]","[65, 108, 160, 192]",946.431070,2513.719321,1842.928815,...,"[[108, 255, 255, 255, 255, 255, 255, 255, 255,...","[959.8682768447037, 932.3540397809779, 956.563...","[2520.429461171158, 2513.32726680122, 2513.887...","[1790.4691179671236, 1810.1847167553412, 1843....","[4.401288032531738, 3.4248151779174805, 6.0768...","[1668.482624053955, 1200.2781829833984, 1382.2...","[40.84706383638798, 34.6450311442117, 37.17867...","[15.9075867269207, 17.337984487091266, 16.7249...","[0.7782848354282844, 0.7714387811377484, 0.766...","[0.9531799397351323, 0.9644180527309677, 0.959..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[42, 91, 134, 167, 197]","[30, 74, 124, 164, 200]","[53, 95, 139, 176, 200]",970.986060,2549.214164,1867.290545,...,"[[139, 255, 200, 255, 255, 255, 200, 200, 200,...","[973.7088352366864, 988.6291281732299, 960.476...","[2552.6036107424557, 2544.184710106654, 2515.8...","[1879.207290619318, 1854.215083060793, 1889.31...","[4.290168523788452, 3.413175106048584, 3.37860...","[1247.4132448832195, 1716.2333641052246, 1163....","[35.31873787217232, 41.42744698995129, 34.1107...","[17.17070009841945, 15.785040203316603, 17.472...","[0.7400268498330546, 0.7522756769637766, 0.716...","[0.965815378316412, 0.9569862820205467, 0.9675..."
4,Lena.png,2,mMasi_entropy,max,"[135, 191]","[79, 146]","[99, 149]",884.842049,2205.638673,2100.754813,...,"[[149, 149, 149, 149, 149, 149, 149, 149, 149,...","[734.1293125340219, 786.1062875374504, 817.381...","[2189.5170537402673, 2017.6845734980893, 2320....","[2095.3782138244837, 2038.373394062458, 2135.7...","[3.992122173309326, 3.1927945613861084, 4.0631...","[2917.1547635396323, 2662.5542526245117, 2176....","[54.010691196647656, 51.599944308347, 46.65502...","[13.481208905503227, 13.87781895078241, 14.752...","[0.5642438135339777, 0.5512943070098956, 0.611...","[0.8491072963083409, 0.8530829463382573, 0.880..."
5,Lena.png,3,mMasi_entropy,max,"[107, 157, 203]","[43, 105, 157]","[56, 118, 160]",972.743130,2397.751089,2203.301516,...,"[[160, 160, 160, 160, 160, 118, 160, 160, 160,...","[1037.9107672144778, 1007.5301782366654, 997.1...","[2225.5318071747383, 2493.072801531027, 2384.7...","[2231.888308994461, 2102.7573791481386, 2182.7...","[3.4597883224487305, 4.154113054275513, 3.9225...","[1867.5831871032715, 1266.5121421813965, 1870....","[43.21554335078146, 35.588089892285545, 43.246...","[15.4180040532275

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/SMA/SMA_masi_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/SMA/SMA_masi_RGB_dict.pkl', 'wb'))